In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import  layers, Sequential, Input
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import load_img
 
import os, shutil,cv2
import numpy as np
import pandas as pd
from types import SimpleNamespace
from matplotlib import pyplot as plt

In [ ]:
conf = SimpleNamespace(
    root_dir = "/kaggle/input/brain-tumor-mri-dataset/",
    work_dir = "/kaggle/working/data/",
    train_dir = "/kaggle/working/data/train/",
    test_dir = "/kaggle/working/data/test/",
    val_dir = "/kaggle/working/data/val/",
)

train_conf = SimpleNamespace(
    img_shape = (384, 384),
    batch_size = 32,
    epochs = 5
)

In [ ]:
# EDA

def plot_catg_count(path, name=""):
    fc = 0
    cl_name = []
    cl_count = []
    for d in os.listdir(path):
        #print(f"{d} -- {len(os.listdir(train_path + d))}")
        cl_name.append(d)
        cl_count.append(len(os.listdir(path + d)))
        fc += len(os.listdir(path + d))

    print(f"Total files: {fc}")
    plt.bar(cl_name, cl_count)
    plt.title(name)
    

plot_catg_count(conf.root_dir + "Training/", name="Raw dataset")

# **Making Dataset**

In [ ]:
# Make train, val and test set

def make_work_dir():
    os.makedirs(conf.work_dir, exist_ok=True)    
    os.makedirs(conf.train_dir, exist_ok=True)
    os.makedirs(conf.test_dir, exist_ok=True)
    os.makedirs(conf.val_dir, exist_ok=True)


def make_split(split_size=[0.7, 0.2, 0.1], load_lim=None):
    make_work_dir()
    
    raw_train_path = conf.root_dir + "Training/" 
    for d in os.listdir(raw_train_path):
        sub_dir = f"{raw_train_path}{d}/"
        files = os.listdir(sub_dir)
        files = files[:load_lim]
        file_count = len(files)
        
        for idx, fname in enumerate(files):
            if idx < int(file_count * split_size[0]):
                os.makedirs(f"{conf.train_dir}{d}/", exist_ok=True)
                f_new_path = f"{conf.train_dir}{d}/{fname}"
            elif idx < int(file_count * (split_size[0] + split_size[1])):
                os.makedirs(f"{conf.val_dir}{d}/", exist_ok=True)
                f_new_path = f"{conf.val_dir}{d}/{fname}"
            else:
                os.makedirs(f"{conf.test_dir}{d}/", exist_ok=True)
                f_new_path = f"{conf.test_dir}{d}/{fname}"
                
            f_curr_path = f"{sub_dir}{fname}"
            shutil.copyfile(f_curr_path, f_new_path)

In [ ]:
make_split()

In [ ]:
train_set = image_dataset_from_directory(
    conf.train_dir,
    batch_size=train_conf.batch_size,
    image_size=train_conf.img_shape,
    shuffle=True,
)

val_set = image_dataset_from_directory(
    conf.val_dir,
    batch_size=train_conf.batch_size,
    image_size=train_conf.img_shape,
    shuffle=True,
)

test_set = image_dataset_from_directory(
    conf.test_dir,
    batch_size=train_conf.batch_size,
    image_size=train_conf.img_shape,
    shuffle=True,
)

In [ ]:
plot_catg_count(conf.train_dir, "Training Images")
plot_catg_count(conf.val_dir, "Validation Images")
plot_catg_count(conf.test_dir, "Test Images")

# **InceptionV3 based model**

In [ ]:
class InceptionV3BasedModel:
    def __init__(self, model_inp_shape, dense_units=[256, 128, 64, 32, 4]):
        self.input_shape = model_inp_shape
        
        self.input = Input(shape=self.input_shape)
        self.dense_units = dense_units
        self.base_learning_rate = 0.0001
        
        self.dense_layers = []
        self.output_layer = layers.Dense(4, activation="softmax", name="output_layer")
        
        self.data_augmentation = Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomRotation(0.2),
        ])
        
        # Setting inception v3
        self.preprocess_input = keras.applications.inception_v3.preprocess_input
        self.inception_v3 = tf.keras.applications.InceptionV3(input_shape=model_inp_shape, include_top=False, weights='imagenet')
        self.inception_v3.trainable = False
        self.inception_summary = self.inception_v3.summary # call for summary

        
    def build_layers(self):
        for idx, d_c in enumerate(self.dense_units):
            self.dense_layers.append(layers.Dense(d_c, activation="relu", name=f"dense_{idx}"))

    def make_model(self):
        self.build_layers()
        inputs = self.input

        x = self.data_augmentation(inputs)
        x = self.preprocess_input(x)
        x = self.inception_v3(x, training=False)

        # Adding dense layers.
        x = layers.Flatten(name="flatten_0")(x)
        for idx, dense_layer in enumerate(self.dense_layers):
            x = dense_layer(x)
            x = layers.Dropout(0.2, name=f"dropout_{idx}")(x)

        outputs = self.output_layer(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs, name="brain-mri-cnn")


    def get_model(self):
        self.make_model()
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=self.base_learning_rate),
            loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
        return self.model

# **Making custom model**

In [ ]:
class BrainNetV01:
    def __init__(self, model_inp_shape, filters=[32, 32, 32, 32, 32, 32, 64, 64, 64, 64, 128, 128, 192, 192, 192, 192, 256, 256],kernel_size=3, dense_units=[128, 64, 32], pool_pres=4):
        """
            Args:
                pool_pres: Pooling layer would be added after pool_pres instance of cnn
        
        """
        self.input_shape = model_inp_shape
        
        self.input = Input(shape=self.input_shape)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dense_units = dense_units
        self.base_learning_rate = 0.0001
        self.pool_pres = pool_pres
        
        self.conv_layers = []
        self.dense_layers = []
        self.max_pools = []
        self.batch_norms = []
        self.conv_acti = []
        self.output_layer = layers.Dense(4, activation="softmax", name="output_layer")
        
        self.data_augmentation = Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomFlip('vertical'),
            layers.RandomRotation(0.2),
        ])
        
        
    def build_layers(self):
        for idx, filter_size in enumerate(self.filters):
            self.conv_layers.append(layers.SeperableConv2D(filter_size, kernel_size=self.kernel_size, name=f"conv_{idx}"))

        for idx, _ in enumerate(self.filters):
            self.batch_norms.append(layers.BatchNormalization(name=f"batch_norm_{idx}"))  
            self.max_pools.append(layers.MaxPooling2D((2, 2), name=f"max_pool_{idx}"))
            self.conv_acti.append(layers.Activation("relu", name=f"activation_{idx}"))

        for idx, d_c in enumerate(self.dense_units):
            self.dense_layers.append(layers.Dense(d_c, activation="relu", name=f"dense_{idx}"))

    def make_model(self):
        self.build_layers()
        inputs = self.input

        # Adding Conv Layers
        for idx, conv_layer in enumerate(self.conv_layers):
            if idx == 0:
                x = conv_layer(inputs)
            else:
                x = conv_layer(x)
                
            # No pooling operation or normalization last layer.
            if idx < len(self.conv_layers) - 1:
                x = self.batch_norms[idx](x)
            
                if idx % self.pool_pres == 0:
                    x = self.max_pools[idx](x)
            x = self.conv_acti[idx](x)

        # Adding dense layers.
        x = layers.Flatten(name="flatten_0")(x)
        for idx, dense_layer in enumerate(self.dense_layers):
            x = dense_layer(x)
            x = layers.Dropout(0.2, name=f"dropout_{idx}")(x)

        outputs = self.output_layer(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs, name="brain-mri-inception_v3")


    def get_model(self):
        self.make_model()
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=self.base_learning_rate),
            loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
        return self.model

In [108]:
class AdvancedBrainNet:
    def __init__(self, model_inp_shape, filters=[16, 32, 48, 64, 96, 128, 164, 192, 256],kernel_size=3, dense_units=[96, 64, 32], res_dur= 2, pool_freq=2):
        """
            Args:
                res_dur: After what percentage of the model would we add resnet at. 0.75 means after 75% of the model, we would add first block's residual connection.
                pool_freq: Koto block pore max pool porbe.
        """
        self.input_shape = model_inp_shape
        
        self.input = Input(shape=self.input_shape)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dense_units = dense_units
        self.base_learning_rate = 0.0001
        self.res_dur = res_dur
        self.pool_freq = pool_freq
        
        self.conv_blocks = []
        self.dense_layers = []
        self.output_layer = layers.Dense(4, activation="softmax", name="output_layer")
        
        self.data_augmentation = Sequential([
            layers.RandomFlip('horizontal'),
            layers.RandomFlip('vertical'),
            layers.RandomRotation(0.2),
        ])
        
   
    def build_layers(self):
        for idx, filter_size in enumerate(self.filters):
            c_block = [
                layers.SeparableConv2D(filter_size, kernel_size=self.kernel_size, name=f"conv__{idx}_0",  padding="same"),
                layers.BatchNormalization(name=f"batch_norm__{idx}_0"),
                layers.Activation("relu", name=f"activation__{idx}_0"),
                layers.SeparableConv2D(filter_size, kernel_size=self.kernel_size, name=f"conv__{idx}_1",  padding="same"),
                layers.BatchNormalization(name=f"batch_norm__{idx}_1"),
                layers.Activation("relu", name=f"activation__{idx}_1"),
                layers.SeparableConv2D(filter_size, kernel_size=self.kernel_size, name=f"conv__{idx}_2",  padding="same"),
                layers.BatchNormalization(name=f"batch_norm__{idx}_2"),
                layers.Activation("relu", name=f"activation__{idx}_2"),
            ]
            self.conv_blocks.append(c_block)
            
        for idx, d_c in enumerate(self.dense_units):
            self.dense_layers.append(layers.Dense(d_c, activation="relu", name=f"dense_{idx}"))

    def make_model(self):
        self.build_layers()
        inputs = self.input

        # Adding Conv Layers
        x_bloc_start = None
        for idx, conv_block in enumerate(self.conv_blocks):
            if idx == 0:
                for lr_idx, cb_layer in enumerate(conv_block):
                    if lr_idx == 0: #only first conv_block's first conv layer will get inputs. All other layer would get x.
                        x = cb_layer(inputs)
                    else:
                        x = cb_layer(x)
            else:
                x_block_start = None
                for lr_idx, cb_layer in enumerate(conv_block):
                    x = cb_layer(x)
                    
                    if lr_idx == 0:
                        x_block_start = x
                    else:
                        lr_idx == len(conv_block) - 1 # last layer of conv, then concat the outputs.
                        x = layers.Add()([x, x_block_start])
                    
                if idx % self.pool_freq == 0:
                    x = layers.MaxPooling2D((2, 2), name=f"max_pool_{idx}")(x)
                
        # Adding dense layers.
        x = layers.Flatten(name="flatten_0")(x)
        for idx, dense_layer in enumerate(self.dense_layers):
            x = dense_layer(x)
            x = layers.Dropout(0.2, name=f"dropout_{idx}")(x)

        outputs = self.output_layer(x)

        self.model = keras.Model(inputs=inputs, outputs=outputs, name="advanced-brain-net")


    def get_model(self):
        self.make_model()
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=self.base_learning_rate),
            loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
        return self.model

### **Training Advanced BrainNet**

In [109]:
model_inp_shape = train_conf.img_shape + (3,)

model_maker = AdvancedBrainNet(model_inp_shape)
model = model_maker.get_model()
keras.utils.plot_model(model, 
                       to_file='advanced_brain_net_model.png', 
                       show_shapes=True,
                        show_layer_names=True,
                        rankdir='LR',
                        expand_nested=True,
                        dpi=320,
                        show_layer_activations=True,
                        show_trainable=True
                    )

In [110]:
model.summary()

Model: "advanced-brain-net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_49      │ (None, 384, 384,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv__0_0           │ (None, 384, 384,  │         91 │ input_layer_49[0… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm__0_0     │ (None, 384, 384,  │         64 │ conv__0_0[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation__0_0     │ (None, 384, 384,  │          0 │ batch_norm__0_0[… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv__0_1           │ (None, 384, 384,  │        416 │ activation__0_0[… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm__0_1     │ (None, 384, 384,  │         64 │ conv__0_1[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation__0_1     │ (None, 384, 384,  │          0 │ batch_norm__0_1[… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv__0_2           │ (None, 384, 384,  │        416 │ activation__0_1[… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm__0_2     │ (None, 384, 384,  │         64 │ conv__0_2[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation__0_2     │ (None, 384, 384,  │          0 │ batch_norm__0_2[… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv__1_0           │ (None, 384, 384,  │        688 │ activation__0_2[… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_norm__1_0     │ (None, 384, 384,  │        128 │ conv__1_0[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_360 (Add)       │ (None, 384, 384,  │          0 │ batch_norm__1_0[… │
│                     │ 32)               │            │ conv__1_0[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation__1_0     │ (None, 384, 384,  │          0 │ add_360[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_361 (Add)       │ (None, 384, 384,  │          0 │ activation__1_0[… │
│                     │ 32)               │            │ conv__1_0[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv__1_1           │ (None, 384, 384,  │      1,344 │ add_361[0][0]     │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_362 (Add)       │ (None, 384, 384,  │          0 │ conv__1_1[0][0],

 Total params: 14,654,231 (55.90 MB)

 Trainable params: 14,648,255 (55.88 MB)

 Non-trainable params: 5,976 (23.34 KB)

In [ ]:
# Loading inception based model
model_inp_shape = train_conf.img_shape + (3,)

incp_model_maker = InceptionV3BasedModel(model_inp_shape)
incp_model = incp_model_maker.get_model()
# print(incp_model.summary())
# print(incp_model_maker.inception_summary())

In [ ]:
model_inp_shape = train_conf.img_shape + (3,)

model_maker = BrainNetV01(model_inp_shape)
model = model_maker.get_model()
model.summary()

In [ ]:
callbacks = keras.callbacks.ModelCheckpoint(
    "brain_mri_4_catg.keras",
    save_best_only=True,
)
callbacks = [callbacks]

# **Training Models**

### Training Inception Based Model

In [ ]:
incp_history = incp_model.fit(train_set, epochs=train_conf.epochs, validation_data=val_set, callbacks=callbacks)

### **Training BrainNet**

In [ ]:
bnet_history = model.fit(train_set, epochs=train_conf.epochs, validation_data=val_set, callbacks=callbacks)

In [ ]:
# Save the weights
checkpoint_path = "/kaggle/working/brain_mri__4catg-{epoch:04d}.weights.h5"
model.save_weights(checkpoint_path.format(epoch=0))

## **Testing on microsoft-cat-vs-dog**

In [ ]:
import tensorflow_datasets as tfds
    
dataset = tfds.load('cats_vs_dogs', batch_size=8)

In [ ]:
train_images = []
val_images = []
test_images = []
for idx, image in enumerate(dataset['train']):
    if idx < 2000:
        train_images.append(image)
    elif idx < 3000:
        val_images.append(image)
    elif idx < 4000:
        test_images.append(image)
    else:
        break

In [ ]:
history = model.fit(train_set, epochs=1, validation_data=val_set, callbacks=callbacks)

In [ ]:
incp_model.get_layer("inception_v3").get_layer('mixed10')

# **GradCAM**

In [ ]:
def grad_cam(model, image, class_index, last_cnn, classifier_layers, isTransferLearning=False):
    """
        TODO: 
            1. Fetch the last conv layer and making a model with the last conv as output.
            2. 
    """
    
    if isTransferLearning:
        r_l, s_l = last_cnn.split(".") # root_layer i.e base_model, s_l i.e. base_model's last layer
        last_conv_layer = model.get_layer(r_l).get_layer(s_l)
        print(last_conv_layer)
    else:
        last_conv_layer = model.get_layer(last_cnn)
    last_conv_layer_model = keras.Model(inputs=model.inputs, outputs=last_conv_layer.output)
    
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    
    for layer_name in classifier_layers:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)
    
    with tf.GradientTape() as tape:
        inputs = image[np.newaxis, ...]
        last_conv_layer_output = last_conv_layer_model(inputs)
        tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = np.argmax(preds[0])
        pred_idx_value = preds[:, top_pred_index]
        
    grads = tape.gradient(pred_idx_value, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]
        
    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.clip(heatmap, 0, np.max(heatmap)) / np.max(heatmap)
    heatmap = cv2.resize(heatmap, train_conf.img_shape)
    
    return heatmap

In [ ]:
model_layers = {
    "inceptionv3o": {'last_cnn': 'inception_v3.mixed10', 'cls_layers' : ["flatten_0", "dense_0", "dense_1", "dense_2", "dense_3"]},
    "brainnet": {'last_cnn':'activation_17', 'cls_layers' : ['flatten_0', 'dense_0', 'dense_1', 'dense_2']}
}

In [ ]:
#pick sample image

sample_image = None

for i in test_set:
    sample_image = i[0][0]
    break

In [ ]:
heatmap = grad_cam(model, sample_image, 1, model_layers['brainnet']['last_cnn'], model_layers['brainnet']['cls_layers'], isTransferLearning=False)   
pred_res = model(np.expand_dims(sample_image, axis=0))
selected_class = np.argmax(pred_res)

plt.imshow(sample_image / 255)
plt.imshow(heatmap, alpha=0.5, cmap="Reds")
plt.title(f"Class: {selected_class}")
plt.savefig("sample_x.jpg")
plt.axis("off")

In [ ]:
#Save everything in  a single output folder.

os.makedirs("outs", exist_ok=True)

for batch_idx, img in enumerate(test_set):
    for img_idx, sample_image in enumerate(img[0]):
        heatmap = grad_cam(model, sample_image, 1) 
        pred_res = model(np.expand_dims(sample_image, axis=0))
        class_idx = np.argmax(pred_res)
        
        plt.imshow(sample_image / 255)
        plt.imshow(heatmap, alpha=0.6, cmap="Reds")
        plt.title(f"Class: {class_idx}")
        plt.savefig(f"outs/batch_{batch_idx}__image_{img_idx}.jpg")
        plt.clf()

In [ ]:
shutil.make_archive("outs", "zip", "/kaggle/working/outs")